In [2]:
# https://man.twcc.ai/@TWCC-III-manual/ryyo0tsuu
def SLURM(cmd):
    ## SLURM 內容, 請修改 ---> Email
    SLURM='''#!/work/c00cjz002/binary/bash5.0/bin/bash
#SBATCH -A MST109178        # 計畫代號
#SBATCH -J CPU_T3_C34       # 工作代號 (標籤, 可自行定義)
#SBATCH -p ngs12G           # 工作區塊 
#SBATCH -c 3                # 使用的CPU核心數
#SBATCH --mem=12g           # 使用的記憶體容量 
#SBATCH --mail-user=summerhill001@gmail.com    # 請修改為您的信向
#SBATCH --mail-type=BEGIN,END                  # 指定送出email時機 可為NONE, BEGIN, END, FAIL, REQUEUE, ALL
#SBATCH -o log/%j.logi      # 執行記錄檔案儲存於log目錄底下
'''
    myCmd = SLURM + cmd
    
    ## 儲存上述內容 SLURM.sh
    import time
    slurm_shell = 'slurm/'+time.strftime("%Y-%m-%d_%H-%M-%S")+'.sh'
    
    f = open(slurm_shell, "w")
    f.write(myCmd)
    f.close()    

    ## 執行SLURM
    #!sbatch SLURM.sh
    jobID=(subprocess.check_output('sbatch '+slurm_shell+' |awk \'{print $4}\'', shell=True,text=True))
    return jobID

## 建立目錄
import subprocess
!mkdir -p slurm
!mkdir -p log

In [3]:
def createCMD(inputFile, saveFolder):
    cmd='''
# -> 輸入變數 (請自行修改或新增)
inputFile='''+ inputFile +''' 
saveFolder='''+ saveFolder +''' 

# -> 切換目錄
cd ${HOME}/ncku_slurm

# -> 環境變數
export MAGPURIFYDB=${HOME}/MAGpurify-db-v1.0

# -> 執行程式
ml libs/singularity/3.7.1

echo "STEP1"
singularity exec \
    /work/c00cjz002/nvidia/magpurify_v2.1.2.sif bash -c \
    "magpurify -h && sleep 5 && echo $(date) > date.txt"
'''
    return cmd

In [6]:
## 指令集
import os
import re
from os import walk
FILES_PATH="/work/c00cjz002/data/Powei20211109/"
files = next(walk(FILES_PATH), (None, None, []))[2]  # [] if no file
pattern = "img$"
for file in files:
    if re.search(pattern, file):
        inputFile=FILES_PATH+file
        saveFolder=os.path.splitext(os.path.basename(inputFile))[0]
        cmd=createCMD(inputFile, saveFolder)
        #print(cmd)
        jobID = SLURM(cmd)
        print(jobID.strip()+' '+saveFolder)

1372792 11477640
1372793 02856573
1372794 12280126
1372795 11477673
1372796 13173476
1372797 08917702
1372798 13277673
1372799 12695924
1372800 12612765
1372801 02322379


In [7]:
## 觀看執行狀況
!squeue -u `whoami` | grep "$jobID\|JOBID" 

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
           1372792    ngs12G CPU_T3_C c00cjz00  R       0:02      1 cpn3857 
           1372793    ngs12G CPU_T3_C c00cjz00  R       0:02      1 cpn3857 
           1372794    ngs12G CPU_T3_C c00cjz00  R       0:02      1 cpn3857 
           1372795    ngs12G CPU_T3_C c00cjz00  R       0:02      1 cpn3857 
           1372796    ngs12G CPU_T3_C c00cjz00  R       0:02      1 cpn3857 
           1372797    ngs12G CPU_T3_C c00cjz00  R       0:02      1 cpn3857 
           1372798    ngs12G CPU_T3_C c00cjz00  R       0:02      1 cpn3857 
           1372799    ngs12G CPU_T3_C c00cjz00  R       0:02      1 cpn3891 
           1372800    ngs12G CPU_T3_C c00cjz00  R       0:02      1 cpn3891 
           1372801    ngs12G CPU_T3_C c00cjz00  R       0:02      1 cpn3891 


In [ ]:
## 刪除JOB
!scancel $jobID

In [15]:
## 全部JOB刪除
!squeue -u `whoami` | grep -v JOBID  | awk '{print $1}' | xargs scancel  # 列出 PID 並砍掉 Process